# Begining of Second Phase 

* sky feature's missing values will be filled after feature engineering on date column
* aim of this notebook is feature engineering and get rid off unwanted columns.

# Import Libraries 

In [1]:
# libraries for EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cufflinks as cf
#Enabling the offline mode for interactive plotting locally
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
cf.go_offline()

#To display the plots
%matplotlib inline
from ipywidgets import interact
import plotly.io as pio

pio.renderers.default = "notebook"

# Ingest Data 

In [2]:
df_birds = pd.read_csv("birds.csv")

df_birds.head()

,opid,operator,atype,remarks,phase_of_flt,ac_mass,num_engs,date,time_of_day,state,height,speed,effect,sky,species,birds_seen,birds_struck
0,AAL,AMERICAN AIRLINES,MD-80,NO DAMAGE,Descent,4.0,2.0,9/30/1990 0:00:00,Night,IL,7000.0,250.0,NaN,No Cloud,UNKNOWN BIRD - MEDIUM,NaN,1
1,USA,US AIRWAYS,FK-28-4000,"2 BIRDS, NO DAMAGE.",Climb,4.0,2.0,11/29/1993 0:00:00,Day,MD,10.0,140.0,None,No Cloud,UNKNOWN BIRD - MEDIUM,2-10,2-10
2,AAL,AMERICAN AIRLINES,B-727-200,NaN,Approach,4.0,3.0,8/13/1993 0:00:00,Day,TN,400.0,140.0,None,Some Cloud,UNKNOWN BIRD - SMALL,2-10,1
3,AAL,AMERICAN AIRLINES,MD-82,NaN,Climb,4.0,2.0,10/7/1993 0:00:00,Day,VA,100.0,200.0,None,Overcast,UNKNOWN BIRD - SMALL,NaN,1
4,AAL,AMERICAN AIRLINES,MD-82,NO DAMAGE,Climb,4.0,2.0,9/25/1993 0:00:00,Day,SC,50.0,170.0,None,Some Cloud,UNKNOWN BIRD - SMALL,2-10,1


In [3]:
df = pd.read_csv("first_phase.csv",parse_dates=["date"])
df.head()

,opid,operator,atype,remarks,phase_of_flt,date,state,height,speed,effect,sky,species,birds_struck
0,USA,US AIRWAYS,FK-28-4000,"2 BIRDS, NO DAMAGE.",Climb,1993-11-29,MD,10.0,140.0,None,No Cloud,UNKNOWN BIRD - MEDIUM,2-10
1,AAL,AMERICAN AIRLINES,B-727-200,NaN,Approach,1993-08-13,TN,400.0,140.0,None,Some Cloud,UNKNOWN BIRD - SMALL,1
2,AAL,AMERICAN AIRLINES,MD-82,NaN,Climb,1993-10-07,VA,100.0,200.0,None,Overcast,UNKNOWN BIRD - SMALL,1
3,AAL,AMERICAN AIRLINES,MD-82,NO DAMAGE,Climb,1993-09-25,SC,50.0,170.0,None,Some Cloud,UNKNOWN BIRD - SMALL,1
4,GFT,GULFSTREAM INTL AIRLINES,BE-99,FLT 714. TIME = 1951Z,Landing Roll,1993-09-20,FL,0.0,40.0,None,Some Cloud,HAWKS,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13116 entries, 0 to 13115
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   opid          13116 non-null  object        
 1   operator      13116 non-null  object        
 2   atype         13116 non-null  object        
 3   remarks       10666 non-null  object        
 4   phase_of_flt  13116 non-null  object        
 5   date          13116 non-null  datetime64[ns]
 6   state         12707 non-null  object        
 7   height        13116 non-null  float64       
 8   speed         13116 non-null  float64       
 9   effect        13116 non-null  object        
 10  sky           11754 non-null  object        
 11  species       13116 non-null  object        
 12  birds_struck  13116 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(10)
memory usage: 1.3+ MB


# Feature Engineering

## date

* Features that I will create "season", "month_of_the_year" 

In [5]:
df.date

0       1993-11-29
1       1993-08-13
2       1993-10-07
3       1993-09-25
4       1993-09-20
           ...    
13111   1997-10-22
13112   1997-06-30
13113   1996-09-19
13114   1997-01-17
13115   1997-11-30
Name: date, Length: 13116, dtype: datetime64[ns]

In [6]:
month_of_the_year = df.date.dt.month_name()
month_of_the_year

0         November
1           August
2          October
3        September
4        September
           ...    
13111      October
13112         June
13113    September
13114      January
13115     November
Name: date, Length: 13116, dtype: object

In [7]:
df["month_of_the_year"] = month_of_the_year

In [8]:
df.month_of_the_year.value_counts()

August       1740
September    1734
October      1605
July         1328
May          1252
November     1075
June          922
April         893
March         770
December      659
January       603
February      535
Name: month_of_the_year, dtype: int64

In [9]:
def seasoner(x):
    if x in ["December","January","February"]:
        return "Winter"
    elif x in ["March","April","May"]:
        return "Spring"
    elif x in ["June","July","August"]:
        return "Summer"
    else:
        return "Fall"

In [10]:
df.month_of_the_year.transform(lambda x : seasoner(x))

0          Fall
1        Summer
2          Fall
3          Fall
4          Fall
          ...  
13111      Fall
13112    Summer
13113      Fall
13114    Winter
13115      Fall
Name: month_of_the_year, Length: 13116, dtype: object

In [11]:
df["season"] = df.month_of_the_year.transform(lambda x : seasoner(x))

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13116 entries, 0 to 13115
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   opid               13116 non-null  object        
 1   operator           13116 non-null  object        
 2   atype              13116 non-null  object        
 3   remarks            10666 non-null  object        
 4   phase_of_flt       13116 non-null  object        
 5   date               13116 non-null  datetime64[ns]
 6   state              12707 non-null  object        
 7   height             13116 non-null  float64       
 8   speed              13116 non-null  float64       
 9   effect             13116 non-null  object        
 10  sky                11754 non-null  object        
 11  species            13116 non-null  object        
 12  birds_struck       13116 non-null  object        
 13  month_of_the_year  13116 non-null  object        
 14  season

* there is no need to keep date column so I will drop

## state

* I will seperate  the states into regions

In [13]:
df.state.value_counts()

CA    1143
FL    1079
TX    1069
NY     837
PA     649
IL     604
NC     415
TN     393
OH     390
NJ     368
DC     366
LA     342
VA     340
KY     340
HI     308
GA     275
MI     266
MO     253
WA     239
AL     211
CT     202
MA     199
WI     189
NE     170
IN     166
MD     154
OR     149
UT     139
CO     130
AK     126
MN     120
IA     115
AR      92
AZ      79
SC      76
OK      67
NV      66
WV      65
MS      61
ME      55
NH      47
KS      47
RI      39
ID      36
NM      32
PR      26
ND      25
SD      25
MT      24
VI      21
VT      16
ON      14
DE      13
QC      11
BC      11
WY       9
AB       3
PI       1
Name: state, dtype: int64

In [14]:
# I found better solution below so this cell stayed just in case
#abbrev_df =pd.read_html("https://abbreviations.yourdictionary.com/articles/state-abbrev.html", header=0)[0]
#abbrev_df

In [15]:
abbrev_df = pd.read_csv("https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv")
abbrev_df

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific
5,Colorado,CO,West,Mountain
6,Connecticut,CT,Northeast,New England
7,District of Columbia,DC,South,South Atlantic
8,Delaware,DE,South,South Atlantic
9,Florida,FL,South,South Atlantic


In [16]:
mask = dict(zip(abbrev_df["State Code"], abbrev_df["Division"]))
mask

{'AK': 'Pacific',
 'AL': 'East South Central',
 'AR': 'West South Central',
 'AZ': 'Mountain',
 'CA': 'Pacific',
 'CO': 'Mountain',
 'CT': 'New England',
 'DC': 'South Atlantic',
 'DE': 'South Atlantic',
 'FL': 'South Atlantic',
 'GA': 'South Atlantic',
 'HI': 'Pacific',
 'IA': 'West North Central',
 'ID': 'Mountain',
 'IL': 'East North Central',
 'IN': 'East North Central',
 'KS': 'West North Central',
 'KY': 'East South Central',
 'LA': 'West South Central',
 'MA': 'New England',
 'MD': 'South Atlantic',
 'ME': 'New England',
 'MI': 'East North Central',
 'MN': 'West North Central',
 'MO': 'West North Central',
 'MS': 'East South Central',
 'MT': 'Mountain',
 'NC': 'South Atlantic',
 'ND': 'West North Central',
 'NE': 'West North Central',
 'NH': 'New England',
 'NJ': 'Middle Atlantic',
 'NM': 'Mountain',
 'NV': 'Mountain',
 'NY': 'Middle Atlantic',
 'OH': 'East North Central',
 'OK': 'West South Central',
 'OR': 'Pacific',
 'PA': 'Middle Atlantic',
 'RI': 'New England',
 'SC': 'Sout

In [17]:
df.state.map(mask).value_counts(dropna=False)

South Atlantic        2783
Pacific               1965
Middle Atlantic       1854
East North Central    1615
West South Central    1570
East South Central    1005
West North Central     755
New England            558
Mountain               515
NaN                    496
Name: state, dtype: int64

In [18]:
df.state = df.state.map(mask)

In [19]:
df.state.value_counts(dropna=False)

South Atlantic        2783
Pacific               1965
Middle Atlantic       1854
East North Central    1615
West South Central    1570
East South Central    1005
West North Central     755
New England            558
Mountain               515
NaN                    496
Name: state, dtype: int64

In [20]:
df.state = df.state.fillna("Unknown")

# Filling missing values in the sky column

In [21]:
df.sky.isnull().sum()

1362

In [22]:
df.loc[df.sky.isnull(), "effect"].value_counts(dropna=False)

None                     957
Precautionary Landing    188
Aborted Take-off         116
Other                     77
Engine Shut Down          24
Name: effect, dtype: int64

In [23]:
modes = df.groupby(["month_of_the_year"]).sky.agg(pd.Series.mode)
modes

month_of_the_year
April          No Cloud
August       Some Cloud
December       No Cloud
February       No Cloud
January        No Cloud
July         Some Cloud
June           No Cloud
March          No Cloud
May            No Cloud
November       No Cloud
October        No Cloud
September      No Cloud
Name: sky, dtype: object

In [24]:
df.loc[df.sky.isnull()]

,opid,operator,atype,remarks,phase_of_flt,date,state,height,speed,effect,sky,species,birds_struck,month_of_the_year,season
7,MTR,METROFLIGHT,BA-31 JETSTR,NO DAMAGE REPTD. TIME = 0042Z,Approach,1992-02-29,West South Central,3000.0,135.0,None,NaN,UNKNOWN BIRD,1,February,Winter
9,MTR,METROFLIGHT,BA-31 JETSTR,FLT 759 HIT BIRD OVER THRESHOLD OF RWY. FLT LA...,Approach,1992-08-19,West South Central,300.0,135.0,None,NaN,BLACKBIRDS,1,August,Summer
17,ABX,ABX AIR (was AIRBORNE EXPRESS),DC-9-30,STRIKE JUST ABOVE F/O'S WINDOW. INSPCTN FOUND ...,Landing Roll,1993-07-30,East South Central,0.0,105.0,None,NaN,UNKNOWN BIRD,1,July,Summer
19,DAL,DELTA AIR LINES,B-767,FLT 1424. DAMAGE TO A/C NOT REPTD.,Landing Roll,1994-07-22,Mountain,0.0,105.0,None,NaN,UNKNOWN BIRD,1,July,Summer
43,UAL,UNITED AIRLINES,B-737,FLT 1438. TIME = 1347Z. HIT A BIRD ON DEPTR. N...,Climb,1993-10-14,Pacific,200.0,150.0,None,NaN,UNKNOWN BIRD,1,October,Fall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13085,FDX,FEDEX EXPRESS,B-727,FED EX 2289 STRUCK A BIRD. NO REPTD DAMGE. FLT...,Landing Roll,1998-07-10,West South Central,0.0,105.0,None,NaN,UNKNOWN BIRD,1,July,Summer
13090,DAL,DELTA AIR LINES,B-737-300,FLT 977 SLC-SJC RETD W/O INCID AFTER REPORTING...,Climb,1998-07-15,Mountain,200.0,150.0,Precautionary Landing,NaN,UNKNOWN BIRD,1,July,Summer
13099,NWA,NORTHWEST AIRLINES,A-320,FLT 368 ABTD T/O & RETD TO GATE DUE TO A BIRD ...,Take-off run,1998-03-03,East North Central,0.0,120.0,Aborted Take-off,NaN,UNKNOWN BIRD,1,March,Spring
13104,UNK,UNKNOWN,C-152,"DARK, MOONLESS NIGHT. DOING TOUCH & GO'S & DID...",Landing Roll,1997-05-01,South Atlantic,0.0,105.0,Engine Shut Down,NaN,WHITE-TAILED DEER,1,May,Spring


In [25]:
[modes[i] for i in df.loc[df.sky.isnull()].month_of_the_year]

['No Cloud',
 'Some Cloud',
 'Some Cloud',
 'Some Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'Some Cloud',
 'Some Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'Some Cloud',
 'No Cloud',
 'Some Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'Some Cloud',
 'No Cloud',
 'No Cloud',
 'Some Cloud',
 'No Cloud',
 'No Cloud',
 'Some Cloud',
 'Some Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'Some Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'Some Cloud',
 'Some Cloud',
 'Some Cloud',
 'No Cloud',
 'Some Cloud',
 'No Cloud',
 'Some Cloud',
 'Some Cloud',
 'No Cloud',
 'Some Cloud',
 'No Cloud',
 'Some Cloud',
 'No Cloud',
 'No Cloud',
 'Some Cloud',
 'No Cloud',
 'No Cloud',
 'Some Cloud',
 'No Cloud',
 'No Cloud',
 'No Cloud',
 'Some Cloud',
 'Some Cloud',
 'Some Cloud',
 'Some Cloud',
 'Some Cloud',
 'Some Cloud',
 'No Cloud',
 'Some C

In [26]:
df.loc[df.sky.isnull(),"sky"] = [modes[i] for i in df.loc[df.sky.isnull()].month_of_the_year]

In [27]:
df.isnull().sum()

opid                    0
operator                0
atype                   0
remarks              2450
phase_of_flt            0
date                    0
state                   0
height                  0
speed                   0
effect                  0
sky                     0
species                 0
birds_struck            0
month_of_the_year       0
season                  0
dtype: int64

# Drop unnecessary columns

In [28]:
df.head()

,opid,operator,atype,remarks,phase_of_flt,date,state,height,speed,effect,sky,species,birds_struck,month_of_the_year,season
0,USA,US AIRWAYS,FK-28-4000,"2 BIRDS, NO DAMAGE.",Climb,1993-11-29,South Atlantic,10.0,140.0,None,No Cloud,UNKNOWN BIRD - MEDIUM,2-10,November,Fall
1,AAL,AMERICAN AIRLINES,B-727-200,NaN,Approach,1993-08-13,East South Central,400.0,140.0,None,Some Cloud,UNKNOWN BIRD - SMALL,1,August,Summer
2,AAL,AMERICAN AIRLINES,MD-82,NaN,Climb,1993-10-07,South Atlantic,100.0,200.0,None,Overcast,UNKNOWN BIRD - SMALL,1,October,Fall
3,AAL,AMERICAN AIRLINES,MD-82,NO DAMAGE,Climb,1993-09-25,South Atlantic,50.0,170.0,None,Some Cloud,UNKNOWN BIRD - SMALL,1,September,Fall
4,GFT,GULFSTREAM INTL AIRLINES,BE-99,FLT 714. TIME = 1951Z,Landing Roll,1993-09-20,South Atlantic,0.0,40.0,None,Some Cloud,HAWKS,1,September,Fall


* opid,operator,remarks,date will be dropped for now

In [29]:
df.drop(["opid","operator","remarks","date"], axis=1, inplace=True)

df

,atype,phase_of_flt,state,height,speed,effect,sky,species,birds_struck,month_of_the_year,season
0,FK-28-4000,Climb,South Atlantic,10.0,140.0,None,No Cloud,UNKNOWN BIRD - MEDIUM,2-10,November,Fall
1,B-727-200,Approach,East South Central,400.0,140.0,None,Some Cloud,UNKNOWN BIRD - SMALL,1,August,Summer
2,MD-82,Climb,South Atlantic,100.0,200.0,None,Overcast,UNKNOWN BIRD - SMALL,1,October,Fall
3,MD-82,Climb,South Atlantic,50.0,170.0,None,Some Cloud,UNKNOWN BIRD - SMALL,1,September,Fall
4,BE-99,Landing Roll,South Atlantic,0.0,40.0,None,Some Cloud,HAWKS,1,September,Fall
...,...,...,...,...,...,...,...,...,...,...,...
13111,C-152,Climb,South Atlantic,50.0,65.0,Precautionary Landing,Some Cloud,UNKNOWN BIRD,1,October,Fall
13112,PA-38 TOMAHAWK,Take-off run,South Atlantic,0.0,120.0,Precautionary Landing,No Cloud,WHITE-TAILED DEER,1,June,Summer
13113,MD-80,Approach,East North Central,50.0,140.0,None,Some Cloud,UNKNOWN BIRD - MEDIUM,1,September,Fall
13114,LOCKHEED 188,Climb,New England,50.0,150.0,None,No Cloud,ROCK PIGEON,1,January,Winter


In [30]:
df.to_csv("ready_for_preprocessing.csv",index=False)